In [98]:
import torch
from torch import nn
import torchvision
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import *

In [99]:
train_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
)

Files already downloaded and verified


In [100]:
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
)

Files already downloaded and verified


In [142]:
bsize = 64
learn_rate = 5e-3
epochs = 50

In [102]:
loss = nn.CrossEntropyLoss()

In [132]:
train_load = DataLoader(train_data, batch_size = bsize)
test_load = DataLoader(test_data, batch_size = bsize)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.conv = nn.Sequential(
            # finish this
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.lin = nn.Sequential(
            nn.Linear(6*5*5,out_features=120),
            nn.Linear(in_features=120, out_features=60),
            nn.Linear(in_features=60, out_features=10),
        )
    
    def forward(self, x):
        logits = self.conv(x)
        logits = torch.flatten(logits, 1)
        logits = self.lin(logits)
        return logits

model = NN()

In [139]:
optim = torch.optim.SGD(model.parameters(), lr=learn_rate, momentum=0.9)

In [134]:
def train_loop(dataloader, model, loss_fn, optim):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader, 0):
        optim.zero_grad()

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optim.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}] \n")

In [135]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [136]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_load, model, loss, optim)
    test_loop(test_load, model, loss)
print("Done!")

Epoch 1
-------------------------------
loss: 2.291567  [    0/50000] 

loss: 2.309814  [ 6400/50000] 

loss: 2.313878  [12800/50000] 

loss: 2.285675  [19200/50000] 

loss: 2.306116  [25600/50000] 

loss: 2.302895  [32000/50000] 

loss: 2.305433  [38400/50000] 

loss: 2.304848  [44800/50000] 

Test Error: 
 Accuracy: 10.1%, Avg loss: 2.301326 

Epoch 2
-------------------------------
loss: 2.293695  [    0/50000] 

loss: 2.306598  [ 6400/50000] 

loss: 2.303005  [12800/50000] 

loss: 2.291342  [19200/50000] 

loss: 2.298562  [25600/50000] 

loss: 2.297467  [32000/50000] 

loss: 2.300410  [38400/50000] 

loss: 2.299894  [44800/50000] 

Test Error: 
 Accuracy: 13.5%, Avg loss: 2.295926 

Epoch 3
-------------------------------
loss: 2.295413  [    0/50000] 

loss: 2.304056  [ 6400/50000] 

loss: 2.289994  [12800/50000] 

loss: 2.293719  [19200/50000] 

loss: 2.287383  [25600/50000] 

loss: 2.288297  [32000/50000] 

loss: 2.293324  [38400/50000] 

loss: 2.291736  [44800/50000] 

Test Err